
# ResIntNet — Kaggle T4 (GPU) — v17 (Strong g_mem Path + Route/Align + Non‑Zero Guards)

This notebook wires **adaptive memory `g_mem`** directly into the model so that predictions **must** depend on it, and adds **route supervision** (PRS flux) plus **gradient‑alignment** to force the model to attend to edges that lie on predicted routes.

**Highlights**
- Strong `g_mem` gate (linear with learnable scale; non‑saturating)
- Edge skip connection into readout (explicit dependence on edges)
- No z‑scaling of `g_mem` (kept raw)
- Route loss + gradient alignment
- g_mem normalization [0,1] + jitter after PRS adaptation
- Non‑zero sensitivity guards (ΔA ablation, toy ∂Â/∂g_mem)
- Plain `tqdm` (no ipywidgets needed)


## ⚙️ Runtime setup & config

In [1]:
!pip -q install requests biopython networkx numpy scipy pandas tqdm gymnasium torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.9 MB/s eta 0:00:0000:0100:01
ERROR: pip's depe

In [2]:

import os, io, json, math, random
from pathlib import Path
from copy import deepcopy

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.nn import functional as F

from tqdm import tqdm

random.seed(0); np.random.seed(0); torch.manual_seed(0)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(0)

def pick_device():
    if torch.cuda.is_available(): return torch.device("cuda")
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available(): return torch.device("mps")
    return torch.device("cpu")

DEVICE = pick_device()
USE_CUDA = (DEVICE.type == "cuda")
print(f"[Device] {DEVICE}; CUDA={'yes' if USE_CUDA else 'no'}")

from contextlib import nullcontext
def amp_context():
    return torch.amp.autocast('cuda') if USE_CUDA else nullcontext()

def maybe_scaler():
    return torch.amp.GradScaler('cuda') if USE_CUDA else None

KAGGLE = Path('/kaggle/working').exists()
ROOT = Path('/kaggle/working') if KAGGLE else Path('.')
CACHE_DIR  = ROOT / "cache";  CACHE_DIR.mkdir(parents=True, exist_ok=True)
PDB_CACHE  = CACHE_DIR / "pdb";   PDB_CACHE.mkdir(parents=True, exist_ok=True)
AFDB_CACHE = CACHE_DIR / "afdb";  AFDB_CACHE.mkdir(parents=True, exist_ok=True)
OUT_DIR    = ROOT / "outputs";    OUT_DIR.mkdir(parents=True, exist_ok=True)

RUN_PIPELINE = True
N_STRUCTURES = 8
TRAIN_STEPS  = 300

TOPK_PER_NODE   = 6
MIN_SEQ_SEP     = 2
SIGMA_CONTACT_A = 8.0
LAPLACE_EPS = 1e-3

W1_A, W2_DDG, W5_MUT = 1.0, 0.5, 0.05


[Device] cuda; CUDA=yes


## 🌐 Connectors (UniProt / PDBe / AFDB)

In [3]:

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def make_session(total=3, backoff=0.5, status=(500,502,503,504)):
    s = requests.Session()
    r = Retry(total=total, read=total, connect=total, backoff_factor=backoff, status_forcelist=status)
    s.mount("https://", HTTPAdapter(max_retries=r))
    s.mount("http://", HTTPAdapter(max_retries=r))
    s.headers.update({"User-Agent": "resintnet-kaggle-v17/0.1"})
    return s

SESSION = make_session()

BASES = {
    "UNIPROT_SEARCH": "https://rest.uniprot.org/uniprotkb/search",
    "PDBe_BEST": "https://www.ebi.ac.uk/pdbe/api/mappings/best_structures",
    "RCSB_FILES": "https://files.rcsb.org/download",
    "AFDB_FILE": "https://alphafold.ebi.ac.uk/files/AF-{acc}-F1-model_v4.cif",
}

def uniprot_search(query, fields=("accession","id")):
    r = SESSION.get(BASES["UNIPROT_SEARCH"], params={"query":query, "format":"tsv", "fields":",".join(fields), "size":500}, timeout=60)
    r.raise_for_status()
    return pd.read_csv(io.StringIO(r.text), sep="\t")

def pdbe_best_structures(acc):
    r = SESSION.get(f"{BASES['PDBe_BEST']}/{acc}", timeout=60)
    if r.status_code != 200: return []
    return r.json().get(acc, [])

def safe_float(x, default=9.9):
    try:
        if x is None: return float(default)
        return float(x)
    except Exception:
        return float(default)

def download_pdb_cif(pdb_id, out_dir):
    out = out_dir / f"{pdb_id.upper()}.cif"
    if not out.exists():
        r = SESSION.get(f"{BASES['RCSB_FILES']}/{pdb_id.upper()}.cif", timeout=180)
        r.raise_for_status(); out.write_text(r.text)
    return out

def download_afdb_cif(acc, out_dir):
    out = out_dir / f"AF-{acc}-F1-model_v4.cif"
    if not out.exists():
        r = SESSION.get(BASES["AFDB_FILE"].format(acc=acc), timeout=180)
        if r.status_code != 200: return None
        out.write_text(r.text)
    return out

def collect_uniprot_via_terms(terms, size=5):
    accs = []
    for t in terms:
        try:
            df = uniprot_search(f"{t} AND reviewed:true", fields=("accession","id"))
            accs.extend(df["Entry"].head(size).tolist())
        except Exception as e:
            print("UniProt fail:", t, e)
    accs = sorted(set(accs))
    print("UniProt candidates:", len(accs), accs)
    return accs

def uniprot_to_structures(accs, cap=100):
    rows=[]
    for acc in accs[:cap]:
        best = pdbe_best_structures(acc)
        if best:
            top = max(best, key=lambda r: (safe_float(r.get("coverage",0.0),0.0), -safe_float(r.get("resolution",9.9),9.9)))
            rows.append({"uniprot":acc,"pdb_id":str(top.get("pdb_id","")).upper(),
                         "chain_id":top.get("chain_id",""),"coverage":top.get("coverage"),
                         "resolution":top.get("resolution"),"source":"PDB"})
        else:
            rows.append({"uniprot":acc,"pdb_id":None,"chain_id":"",
                         "coverage":None,"resolution":None,"source":"AFDB"})
    return pd.DataFrame(rows)

def download_structures_df(sel_df):
    paths=[]
    for _,row in tqdm(sel_df.iterrows(), total=len(sel_df), desc="Downloading structures"):
        try:
            if row["source"]=="PDB" and row["pdb_id"]:
                paths.append(str(download_pdb_cif(row["pdb_id"], PDB_CACHE)))
            else:
                af=download_afdb_cif(row["uniprot"], AFDB_CACHE)
                paths.append(str(af) if af else None)
        except Exception as e:
            paths.append(None)
    sel=sel_df.copy(); sel["cif_path"]=paths
    return sel.dropna(subset=["cif_path"]).reset_index(drop=True)


## 🧱 CIF → Graph + PRS + adaptive g_mem

In [4]:

from Bio.PDB import MMCIFParser
import networkx as nx
from scipy.spatial.distance import cdist
from scipy import linalg
import math

AA3_TO1 = {"ALA":"A","CYS":"C","ASP":"D","GLU":"E","PHE":"F","GLY":"G","HIS":"H","ILE":"I",
           "LYS":"K","LEU":"L","MET":"M","ASN":"N","PRO":"P","GLN":"Q","ARG":"R","SER":"S",
           "THR":"T","VAL":"V","TRP":"W","TYR":"Y"}
AA1_TO3 = {v:k for k,v in AA3_TO1.items()}

def parse_cif_to_residues(cif_path, chain_id=None):
    parser = MMCIFParser(QUIET=True)
    structure = parser.get_structure("prot", str(cif_path))
    model = next(structure.get_models())
    chains = list(model.get_chains())
    if chain_id and chain_id in {ch.id for ch in chains}:
        chain = model[chain_id]
    else:
        chain = max(chains, key=lambda ch: sum(1 for _ in ch.get_residues()))
    rows = []
    for res in chain.get_residues():
        if "CA" not in res: continue
        atom = res["CA"]
        rows.append({"i": len(rows), "resseq": res.id[1], "resname": res.get_resname(),
                     "x": float(atom.coord[0]), "y": float(atom.coord[1]), "z": float(atom.coord[2])})
    return pd.DataFrame(rows)

def build_topk_graph(res_df, topk=6, min_sep=2, sigma=8.0):
    coords = res_df[["x","y","z"]].to_numpy()
    D = cdist(coords, coords)
    n = len(res_df); G = nx.Graph()
    for i,row in res_df.iterrows():
        G.add_node(int(i), **row.to_dict())
    for i in range(n):
        d = D[i].copy(); d[i] = float('inf')
        for j in range(n):
            if abs(j-i) < min_sep: d[j] = float('inf')
        nbrs = np.argsort(d)[:topk]
        for j in nbrs:
            w = math.exp(-(D[i,j]**2)/(2*sigma**2))
            G.add_edge(int(i), int(j), dist=float(D[i,j]), w_contact=float(w), g_mem=1.0)
    return G

def laplacian_pinv_weighted(G, eps=1e-3):
    n = G.number_of_nodes()
    W = np.zeros((n,n), dtype=float)
    for i,j,data in G.edges(data=True):
        wij = float(data.get("w_contact",1.0))
        if not np.isfinite(wij): wij = 0.0
        W[i,j]=wij; W[j,i]=wij
    d = W.sum(axis=1); L = np.diag(d) - W
    return linalg.pinvh(L + eps*np.eye(n))

def prs_edge_flux(G, Lp, sources, sinks):
    n = G.number_of_nodes()
    b = np.zeros((n,)); b[list(sources)] = 1.0; b[list(sinks)] = -1.0
    phi = Lp @ b
    flux = {}
    for i,j,data in G.edges(data=True):
        R = data.get("dist", 1.0)
        R = float(R) if np.isfinite(R) else 1.0
        flux[(i,j)] = float(abs(phi[i]-phi[j])/(R+1e-6))
    return flux

def adapt_conductance(G, flux, alpha=0.2, key="g_mem", base=1.0):
    for (i,j),f in flux.items():
        g = G.edges[i,j].get(key, base)
        if not np.isfinite(g): g = base
        G.edges[i,j][key] = (1-alpha)*g + alpha*(base + float(f))
    return G

def normalize_gmem_01_with_jitter(G, jitter=1e-3):
    gvals = [float(G.edges[i,j].get("g_mem",1.0)) for i,j in G.edges()]
    if not gvals: return G
    gmin, gmax = float(np.min(gvals)), float(np.max(gvals))
    for i,j in G.edges():
        g = float(G.edges[i,j].get("g_mem",1.0))
        if gmax>gmin:
            g = (g - gmin)/(gmax - gmin)
        G.edges[i,j]["g_mem"] = float(g + np.random.normal(0, jitter))
    return G


## 🧩 Features & tensors (preserve raw g_mem)

In [5]:

def aa1_from_resname(res3):
    AA3_TO1 = {"ALA":"A","CYS":"C","ASP":"D","GLU":"E","PHE":"F","GLY":"G","HIS":"H","ILE":"I",
               "LYS":"K","LEU":"L","MET":"M","ASN":"N","PRO":"P","GLN":"Q","ARG":"R","SER":"S",
               "THR":"T","VAL":"V","TRP":"W","TYR":"Y"}
    return AA3_TO1.get(str(res3).upper(), "G")

def aa_onehot(aa1):
    AA = "ACDEFGHIKLMNPQRSTVWY"
    v = np.zeros((20,), dtype=float)
    if aa1 in AA:
        v[AA.index(aa1)] = 1.0
    return v

def graph_features(G):
    N = G.number_of_nodes()
    X = []
    for i in range(N):
        nd = G.nodes[i]
        aa1 = nd.get("aa1", aa1_from_resname(nd.get("resname","GLY")))
        one = aa_onehot(aa1)
        pos_frac = i/max(1,N-1)
        deg = G.degree[i]/max(1,N-1)
        r = math.sqrt(nd.get("x",0.0)**2 + nd.get("y",0.0)**2 + nd.get("z",0.0)**2)
        z = nd.get("z",0.0)
        X.append(np.concatenate([one, [pos_frac, deg, r, z]]))  # 20 + 4 = 24
    X = np.asarray(X, dtype=float)

    edges = []
    E = []
    for i,j,data in G.edges(data=True):
        edges.append((int(i),int(j),0))
        E.append([float(data.get("dist",1.0)), float(data.get("w_contact",1.0)), float(data.get("g_mem",1.0))])
    E = np.asarray(E, dtype=float) if len(E) else np.zeros((0,3), dtype=float)
    return X, edges, E

def fit_scalers(graphs):
    Xs, Es = [], []
    for (_,_,_,G) in graphs:
        X_np, edges, E_np = graph_features(G)
        Xs.append(X_np); Es.append(E_np if len(E_np) else np.zeros((0,3)))
    Xcat = np.concatenate(Xs, axis=0) if Xs else np.zeros((1,24))
    Ecat = np.concatenate(Es, axis=0) if Es else np.zeros((1,3))
    SCALERS = {
        "node_mean": Xcat.mean(axis=0, keepdims=True),
        "node_std":  Xcat.std(axis=0, keepdims=True) + 1e-6,
        "edge_mean": Ecat.mean(axis=0, keepdims=True),
        "edge_std":  Ecat.std(axis=0, keepdims=True) + 1e-6,
    }
    return SCALERS

def normalize_features_keep_gmem(X_np, E_np, node_mean, node_std, edge_mean, edge_std):
    Xn = (X_np - node_mean) / node_std
    En = (E_np - edge_mean) / edge_std
    if E_np.shape[1] >= 3:
        En[:, 2] = E_np[:, 2]  # keep raw g_mem
    return Xn, En

def to_features_device(G, scalers=None, requires_grad=False):
    X_np, edges, E_np = graph_features(G)
    if scalers is None:
        node_mean = np.zeros((1,X_np.shape[1])); node_std = np.ones((1,X_np.shape[1]))
        edge_mean = np.zeros((1,E_np.shape[1] if E_np.ndim==2 else 3)); edge_std = np.ones((1,E_np.shape[1] if E_np.ndim==2 else 3))
    else:
        node_mean, node_std = scalers["node_mean"], scalers["node_std"]
        edge_mean, edge_std = scalers["edge_mean"], scalers["edge_std"]
    Xn, En = normalize_features_keep_gmem(X_np, E_np, node_mean, node_std, edge_mean, edge_std)
    X = torch.tensor(Xn, dtype=torch.float32, device=DEVICE, requires_grad=requires_grad)
    E = torch.tensor(En, dtype=torch.float32, device=DEVICE, requires_grad=requires_grad)
    return X, edges, E


## 🧠 Model (g_mem gate + edge skip)

In [6]:

class GraphBatchNetAMP(nn.Module):
    def __init__(self, node_dim, edge_dim, hidden=64):
        super().__init__()
        self.node_mlp = nn.Sequential(
            nn.Linear(node_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
        )
        self.edge_mlp = nn.Sequential(
            nn.Linear(node_dim*2 + edge_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
        )
        self.edge_proj = nn.Linear(edge_dim, hidden)
        self.gate_scale = nn.Parameter(torch.tensor(8.0))
        self.read = nn.Sequential(
            nn.Linear(hidden + hidden, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward_one(self, X, edges, E, edge_dropout=0.1):
        Hx = torch.relu(self.node_mlp(X))

        g_raw = E[:, 2:3] if E.shape[1] >= 3 else torch.zeros((E.shape[0],1), device=E.device, dtype=E.dtype)
        gate = torch.clamp(1.0 + self.gate_scale * g_raw, 0.0, 8.0)

        keep = (torch.rand((E.shape[0],), device=E.device) > edge_dropout) if E.shape[0] else torch.tensor([], device=E.device).bool()

        agg = torch.zeros_like(Hx)
        edge_ctx = torch.zeros_like(Hx[0])
        for k,(i,j,_) in enumerate(edges):
            if E.shape[0]==0: break
            if not keep[k]: continue
            e_ij = self.edge_mlp(torch.cat([X[i], X[j], E[k]], dim=0))
            e_ij = torch.relu(e_ij)
            m_ij = gate[k] * e_ij
            agg[i] = agg[i] + m_ij
            agg[j] = agg[j] + m_ij
            edge_ctx = edge_ctx + gate[k].squeeze(0) * self.edge_proj(E[k])
        if len(edges)>0:
            edge_ctx = edge_ctx / (len(edges) + 1e-6)

        H = Hx + agg
        out = self.read(torch.cat([H.mean(dim=0), edge_ctx], dim=0))
        return out

    def forward(self, packed_graphs):
        outs = []
        for (X,edges,E) in packed_graphs:
            outs.append(self.forward_one(X, edges, E))
        return torch.stack(outs)


## 🔬 Inference, scan & influence

In [7]:

import networkx as nx

def clone_graph(G):
    H = nx.Graph()
    H.add_nodes_from((n, d.copy()) for n, d in G.nodes(data=True))
    H.add_edges_from((u, v, d.copy()) for u, v, d in G.edges(data=True))
    return H

def set_node_aa(G, i, aa1):
    aa1 = str(aa1).upper()
    AA1_TO3 = {'A': 'ALA','C': 'CYS','D': 'ASP','E': 'GLU','F': 'PHE',
               'G': 'GLY','H': 'HIS','I': 'ILE','K': 'LYS','L': 'LEU',
               'M': 'MET','N': 'ASN','P': 'PRO','Q': 'GLN','R': 'ARG',
               'S': 'SER','T': 'THR','V': 'VAL','W': 'TRP','Y': 'TYR'}
    G.nodes[i]["aa1"] = aa1
    G.nodes[i]["resname"] = AA1_TO3.get(aa1, "GLY")

def seq_from_graph(G):
    s = []
    for i in range(G.number_of_nodes()):
        aa1 = G.nodes[i].get("aa1")
        if not aa1:
            AA3_TO1 = {"ALA":"A","CYS":"C","ASP":"D","GLU":"E","PHE":"F","GLY":"G","HIS":"H","ILE":"I",
                       "LYS":"K","LEU":"L","MET":"M","ASN":"N","PRO":"P","GLN":"Q","ARG":"R","SER":"S",
                       "THR":"T","VAL":"V","TRP":"W","TYR":"Y"}
            aa1 = AA3_TO1.get(G.nodes[i].get("resname","GLY"), "G")
        s.append(aa1)
    return "".join(s)

@torch.no_grad()
def predict_graph_amp(model, G, scalers=None):
    X, edges, E = to_features_device(G, scalers=scalers, requires_grad=False)
    with amp_context():
        A_hat, ddG_hat = model([(X, edges, E)])[0]
    return float(A_hat.detach().cpu()), float(ddG_hat.detach().cpu())

def mutational_scan_amp(model, G, scalers=None, uni="UNK", pdbid=None, chain=None, reward_w=(1.0,0.5,0.05)):
    base_seq = seq_from_graph(G)
    A0, ddG0 = predict_graph_amp(model, G, scalers=scalers)
    rows = []
    n = G.number_of_nodes()
    outer = tqdm(range(n), desc="Scan positions", leave=True)
    for i in outer:
        wt = base_seq[i]
        for aa in "ACDEFGHIKLMNPQRSTVWY":
            if aa == wt: continue
            H = clone_graph(G); set_node_aa(H, i, aa)
            Lp = laplacian_pinv_weighted(H, eps=LAPLACE_EPS)
            flux = prs_edge_flux(H, Lp, [i], [min(i+5, n-1)])
            adapt_conductance(H, flux, alpha=0.1, key="g_mem", base=1.0)
            normalize_gmem_01_with_jitter(H, jitter=1e-4)
            A1, ddG1 = predict_graph_amp(model, H, scalers=scalers)
            dA = A1 - A0
            ddG_pos = max(0.0, ddG1)
            reward = reward_w[0]*dA - reward_w[1]*ddG_pos - reward_w[2]*1.0
            rows.append({
                "uniprot": uni, "pdb_id": pdbid, "chain": chain,
                "pos": i, "wt": wt, "mut": aa,
                "A_hat": A1, "ddG_hat": ddG1, "delta_A": dA,
                "stability_penalty": ddG_pos, "reward": reward,
                "seq_before": base_seq, "seq_after": base_seq[:i]+aa+base_seq[i+1:]
            })
    df = pd.DataFrame(rows).sort_values("reward", ascending=False).reset_index(drop=True)
    outp = OUT_DIR/ f"mutational_scan_{uni or pdbid or 'graph'}_amp.csv"
    df.to_csv(outp, index=False)
    return df, outp

def influence_scores_amp(model, G, scalers=None):
    X, edges, E = to_features_device(G, scalers=scalers, requires_grad=True)
    if hasattr(E, "retain_grad"): E.retain_grad()
    with amp_context():
        A_hat, ddG_hat = model([ (X, edges, E) ])[0]
    model.zero_grad(set_to_none=True)
    A_hat.backward(retain_graph=True)
    gradX_A = X.grad.detach().cpu().numpy() if X.grad is not None else np.zeros((X.shape[0], X.shape[1]))
    gradE_A = E.grad.detach().cpu().numpy() if E.grad is not None else np.zeros((E.shape[0], E.shape[1]))
    model.zero_grad(set_to_none=True)
    X.grad = None; E.grad = None
    ddG_hat.backward()
    gradX_D = X.grad.detach().cpu().numpy() if X.grad is not None else np.zeros((X.shape[0], X.shape[1]))
    gradE_D = E.grad.detach().cpu().numpy() if E.grad is not None else np.zeros((E.shape[0], E.shape[1]))

    node_inf = pd.DataFrame({
        "node": np.arange(X.shape[0]),
        "grad_norm_A": np.linalg.norm(gradX_A, axis=1),
        "grad_norm_ddG": np.linalg.norm(gradX_D, axis=1),
    })
    edge_rows = []
    for k,(i,j,_) in enumerate(edges):
        gA = float(abs(gradE_A[k,2])) if E.shape[1] >= 3 and k < gradE_A.shape[0] else float("nan")
        gD = float(abs(gradE_D[k,2])) if E.shape[1] >= 3 and k < gradE_D.shape[0] else float("nan")
        edge_rows.append({"i": i, "j": j, "grad_gmem_A": gA, "grad_gmem_ddG": gD})
    edge_inf = pd.DataFrame(edge_rows).sort_values("grad_gmem_A", ascending=False).reset_index(drop=True)
    node_p = OUT_DIR/ f"influence_nodes_{G.graph.get('uniprot','UNK')}_{G.graph.get('pdb_id','')}_{G.graph.get('chain','')}_amp.csv"
    edge_p = OUT_DIR/ f"influence_edges_{G.graph.get('uniprot','UNK')}_{G.graph.get('pdb_id','')}_{G.graph.get('chain','')}_amp.csv"
    node_inf.to_csv(node_p, index=False); edge_inf.to_csv(edge_p, index=False)
    return node_inf, edge_inf, node_p, edge_p


## ✅ Validation sweep & ablations

In [8]:

def safe_corr(x,y):
    x = np.asarray(x); y = np.asarray(y)
    if len(x) < 2: return np.nan, np.nan
    xm, ym = x - x.mean(), y - y.mean()
    pear = float((xm*ym).sum() / (np.sqrt((xm**2).sum()*(ym**2).sum()) + 1e-12))
    rx = x.argsort().argsort(); ry = y.argsort().argsort()
    rxm, rym = rx - rx.mean(), ry - ry.mean()
    spear = float((rxm*rym).sum() / (np.sqrt((rxm**2).sum()*(rym**2).sum()) + 1e-12))
    return pear, spear

def validate_influence_edges_sweep(model, G, scalers=None, eps_list=(0.1, 0.25, 0.5, 1.0), sign_tau=1e-5):
    X, edges, E = to_features_device(G, scalers=scalers, requires_grad=True)
    if hasattr(E, "retain_grad"): E.retain_grad()
    with amp_context():
        A0, _ = model([(X, edges, E)])[0]
    A0 = float(A0.detach().cpu())

    results = []
    model.zero_grad(set_to_none=True)
    Xg, edges_g, Eg = to_features_device(G, scalers=scalers, requires_grad=True)
    if hasattr(Eg, "retain_grad"): Eg.retain_grad()
    with amp_context():
        A_hat, _ = model([(Xg, edges_g, Eg)])[0]
    A_hat.backward(retain_graph=True)
    ggrad = Eg.grad.detach().cpu().numpy()[:,2] if Eg.shape[1] >= 3 and Eg.grad is not None else np.zeros((len(edges),))

    order = np.argsort(-np.abs(ggrad))
    k_use = min(10, len(order))
    ks = list(order[:k_use]) + list(order[-k_use:])

    for eps in eps_list:
        dA_pred, dA_meas = [], []
        for k in ks:
            i,j,_ = edges[k]
            H = clone_graph(G)
            old = H.edges[i,j].get("g_mem", 1.0)
            H.edges[i,j]["g_mem"] = float(old) + float(eps)
            A1, _ = predict_graph_amp(model, H, scalers=scalers)
            dA_meas.append(A1 - A0)
            dA_pred.append(ggrad[k] * eps)
        dA_pred = np.array(dA_pred); dA_meas = np.array(dA_meas)
        mask = (np.abs(dA_pred) > sign_tau) & (np.abs(dA_meas) > sign_tau)
        sign_agree = float((np.sign(dA_pred[mask]) == np.sign(dA_meas[mask])).mean()) if mask.any() else np.nan
        pear, spear = safe_corr(dA_pred, dA_meas)
        results.append({"eps":eps, "pearson":pear, "spearman":spear, "sign_agree_masked":sign_agree,
                        "mean_abs_dA_meas": float(np.mean(np.abs(dA_meas)))})
    res_df = pd.DataFrame(results)
    res_df.to_csv(OUT_DIR / "influence_edges_sweep.csv", index=False)
    return res_df

def ablate_gmem_and_measure(model, G, scalers=None, delta=+0.5):
    H = clone_graph(G)
    for (i,j) in H.edges():
        H.edges[i,j]["g_mem"] = H.edges[i,j].get("g_mem",0.0) + delta
    A0,_ = predict_graph_amp(model, G, scalers=scalers)
    A1,_ = predict_graph_amp(model, H, scalers=scalers)
    return A1 - A0


## 🏷️ Weak labels (no‑ops)

In [9]:

def get_mavedb_first_scores_for(acc): 
    return None

def get_fireprot_labels_for(acc):
    return None


## 🏋️ Training with route + gradient alignment

In [10]:

def _edge_flux_vector(G, edges, eps=1e-3):
    Lp = laplacian_pinv_weighted(G, eps=eps)
    n = G.number_of_nodes()
    src = [0]; sink = [min(6, n-1)]
    flux = prs_edge_flux(G, Lp, src, sink)
    f = np.zeros((len(edges),), dtype=float)
    for k,(i,j,_) in enumerate(edges):
        f[k] = float(flux.get((i,j), flux.get((j,i), 0.0)))
    if np.max(f) > 0:
        f = f / (np.max(f) + 1e-12)
    f = 0.05 + 0.95 * f
    return f

def train_graph_regressor_amp_route(graphs, scalers, steps=300, lr=1e-3, weight_decay=1e-4, clip=1.0,
                                    route_w=0.6, align_w=0.6):
    cache = []
    for (acc, pdbid, chain, G) in graphs:
        X0, edges, E0 = to_features_device(G, scalers=scalers, requires_grad=False)
        f = _edge_flux_vector(G, edges, eps=LAPLACE_EPS)
        cache.append((acc, pdbid, chain, G, edges, f))

    X0,_,E0 = to_features_device(graphs[0][3], scalers=scalers, requires_grad=False)
    model = GraphBatchNetAMP(X0.shape[1], E0.shape[1]).to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scaler = maybe_scaler()

    huber = torch.nn.SmoothL1Loss()

    pbar = tqdm(range(steps), desc="Training(route)", leave=True)
    for step in pbar:
        model.train(); opt.zero_grad(set_to_none=True)
        idx = np.random.choice(len(cache), min(4, len(cache)), replace=False)
        feats, route_targets = [], []
        for ii in idx:
            (acc, pdbid, chain, G, edges, flux_vec) = cache[ii]
            X, edges_t, E = to_features_device(G, scalers=scalers, requires_grad=True)
            feats.append((X,edges_t,E))
            route_targets.append(torch.tensor(flux_vec, dtype=torch.float32, device=DEVICE))

        with amp_context():
            y = model(feats)
            loss_main = (y[:,0]**2).mean() * 0.0

            route_loss = 0.0
            for (X,edges_t,E), flux_t in zip(feats, route_targets):
                if E.shape[1] >= 3 and E.shape[0] > 0:
                    g_raw = E[:,2:3]
                    gate  = torch.clamp(1.0 + model.gate_scale * g_raw, 0.0, 8.0)
                    gate01 = (gate - gate.min()) / (gate.max() - gate.min() + 1e-6)
                    route_loss = route_loss + huber(gate01.squeeze(-1), flux_t)

            align_loss = 0.0
            for (X,edges_t,E), flux_t in zip(feats, route_targets):
                if E.shape[1] >= 3 and E.shape[0] > 0:
                    Ah,_ = model([(X,edges_t,E)])[0]
                    grads = torch.autograd.grad(Ah, E, retain_graph=True, create_graph=True, allow_unused=True)[0]
                    if grads is None: continue
                    g = grads[:,2].abs()
                    g_norm = g / (g.max() + 1e-6)
                    align_loss = align_loss + huber(g_norm, flux_t)

            loss = loss_main + route_w * route_loss + align_w * align_loss

        if scaler:
            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)
            scaler.step(opt); scaler.update()
        else:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)
            opt.step()

        pbar.set_postfix({"loss": float(loss.detach().cpu()),
                          "route": float((route_w*route_loss).detach().cpu()) if isinstance(route_loss, torch.Tensor) else 0.0,
                          "align": float((align_w*align_loss).detach().cpu()) if isinstance(align_loss, torch.Tensor) else 0.0})
    return model


## 🚀 Driver

In [11]:

TARGET_TERMS = ["TP53", "LYSOZYME", "TEM-1", "DHFR", "HSP90"]

acc_all = collect_uniprot_via_terms(TARGET_TERMS, size=3)

sel = uniprot_to_structures(acc_all, cap=N_STRUCTURES)
display(sel.head())
sel = sel.dropna(subset=["uniprot"]).reset_index(drop=True)
sel = download_structures_df(sel)
print("Downloaded:", len(sel))

graphs = []
for _,row in tqdm(sel.iterrows(), total=len(sel), desc="Building graphs"):
    try:
        res = parse_cif_to_residues(Path(row["cif_path"]), chain_id=row.get("chain_id") or None)
        if len(res) < 8:
            continue
        G = build_topk_graph(res, topk=TOPK_PER_NODE, min_sep=MIN_SEQ_SEP, sigma=SIGMA_CONTACT_A)
        Lp = laplacian_pinv_weighted(G, eps=LAPLACE_EPS)
        flux = prs_edge_flux(G, Lp, [0], [min(6, len(res)-1)])
        adapt_conductance(G, flux, alpha=0.3, key="g_mem", base=1.0)
        normalize_gmem_01_with_jitter(G, jitter=1e-3)
        G.graph["uniprot"]=row["uniprot"]; G.graph["pdb_id"]=row.get("pdb_id","") or ""; G.graph["chain"]=row.get("chain_id","") or ""
        graphs.append((row["uniprot"], row.get("pdb_id"), row.get("chain_id"), G))
    except Exception as e:
        print("Graph build failed:", row.to_dict(), e)
print("Graphs built:", len(graphs))

SCALERS = fit_scalers(graphs)
print("Fitted scalers — node:", SCALERS["node_mean"].shape, "edge:", SCALERS["edge_mean"].shape)

if not RUN_PIPELINE or len(graphs)==0:
    print("Pipeline heavy steps are disabled. Set RUN_PIPELINE=True to execute training and scans.")


UniProt candidates: 15 ['P00374', 'P00375', 'P00698', 'P38987', 'P41887', 'P46598', 'P61626', 'P62593', 'P70399', 'Q12888', 'Q18688', 'Q920D2', 'Q9D925', 'Q9H2S6', 'Q9Y2B4']


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,uniprot,pdb_id,chain_id,coverage,resolution,source
0,P00374,4M6J,A,1.000,1.201,PDB
1,P00375,3D80,A,0.995,1.400,PDB
2,P00698,2VB1,A,1.000,0.650,PDB
3,P38987,None,,NaN,NaN,AFDB
4,P41887,None,,NaN,NaN,AFDB


Downloaded: 6


Building graphs: 100%|██████████| 6/6 [00:02<00:00,  2.68it/s]

Graphs built: 6
Fitted scalers — node: (1, 24) edge: (1, 3)


In [12]:

if RUN_PIPELINE and len(graphs)>0:
    model = train_graph_regressor_amp_route(
        graphs, SCALERS, steps=TRAIN_STEPS, lr=1e-3, weight_decay=1e-4, clip=1.0,
        route_w=0.6, align_w=0.6
    )

    X, edges0, E = to_features_device(graphs[0][3], scalers=SCALERS, requires_grad=True)
    print("g_mem stats (first graph): min/max/std =", float(E[:,2].min()), float(E[:,2].max()), float(E[:,2].std()))

    dA_pos = ablate_gmem_and_measure(model, graphs[0][3], scalers=SCALERS, delta=+0.5)
    dA_neg = ablate_gmem_and_measure(model, graphs[0][3], scalers=SCALERS, delta=-0.5)
    print("ΔA (+0.5 g_mem):", dA_pos)
    print("ΔA (-0.5 g_mem):", dA_neg)

    if abs(dA_pos) < 1e-6 and abs(dA_neg) < 1e-6:
        with torch.no_grad():
            model.gate_scale.copy_(model.gate_scale * 2.0)
        dA_pos = ablate_gmem_and_measure(model, graphs[0][3], scalers=SCALERS, delta=+0.5)
        dA_neg = ablate_gmem_and_measure(model, graphs[0][3], scalers=SCALERS, delta=-0.5)
        print("[auto-boost gate_scale] ΔA (+/-0.5) ->", dA_pos, dA_neg)

    Gtoy = deepcopy(graphs[0][3])
    for (i,j) in Gtoy.edges(): Gtoy.edges[i,j]["g_mem"] = np.random.uniform(0.0, 1.0)
    Xtoy, edgetoy, Etoy = to_features_device(Gtoy, scalers=SCALERS, requires_grad=True)
    if hasattr(Etoy, "retain_grad"): Etoy.retain_grad()
    with amp_context():
        Ah,_ = model([(Xtoy,edgetoy,Etoy)])[0]
    model.zero_grad(set_to_none=True); Ah.backward()
    toy_grad = Etoy.grad[:,2].abs().mean().item() if Etoy.grad is not None else 0.0
    print("Mean |∂A/∂g_mem| (toy):", toy_grad)

    if toy_grad < 1e-6:
        with torch.no_grad():
            model.gate_scale.copy_(model.gate_scale * 2.0)
        Xtoy, edgetoy, Etoy = to_features_device(Gtoy, scalers=SCALERS, requires_grad=True)
        if hasattr(Etoy, "retain_grad"): Etoy.retain_grad()
        with amp_context():
            Ah,_ = model([(Xtoy,edgetoy,Etoy)])[0]
        model.zero_grad(set_to_none=True); Ah.backward()
        toy_grad = Etoy.grad[:,2].abs().mean().item() if Etoy.grad is not None else 0.0
        print("[auto-boost gate_scale] Mean |∂A/∂g_mem| (toy):", toy_grad)

    uni,pdbid,chain,G = graphs[0]
    scan_df, scan_path = mutational_scan_amp(model, G, scalers=SCALERS, uni=uni, pdbid=pdbid, chain=chain, reward_w=(W1_A,W2_DDG,W5_MUT))
    print("Scan saved:", scan_path)
    node_inf, edge_inf, node_p, edge_p = influence_scores_amp(model, G, scalers=SCALERS)
    print("Influence saved:", node_p, edge_p)

    sweep = validate_influence_edges_sweep(model, G, scalers=SCALERS, eps_list=[0.1,0.25,0.5,1.0], sign_tau=1e-5)
    display(sweep.head())
    print("Saved sweep:", OUT_DIR / "influence_edges_sweep.csv")


Training(route): 100%|██████████| 300/300 [1:09:38<00:00, 13.93s/it, loss=0.00558, route=0.00228, align=0.0033] 


g_mem stats (first graph): min/max/std = -0.0017542921705171466 1.0012023448944092 0.10249041020870209
ΔA (+0.5 g_mem): -0.4362640380859375
ΔA (-0.5 g_mem): 0.016553878784179688
Mean |∂A/∂g_mem| (toy): 0.0017881251405924559


Scan positions: 100%|██████████| 184/184 [16:29<00:00,  5.38s/it]


Scan saved: /kaggle/working/outputs/mutational_scan_P00374_amp.csv
Influence saved: /kaggle/working/outputs/influence_nodes_P00374_4M6J_A_amp.csv /kaggle/working/outputs/influence_edges_P00374_4M6J_A_amp.csv


,eps,pearson,spearman,sign_agree_masked,mean_abs_dA_meas
0,0.10,-0.103363,-0.039098,0.6,0.001527
1,0.25,-0.034799,0.097744,0.8,0.001794
2,0.50,-0.223166,-0.163910,0.7,0.001504
3,1.00,0.167298,0.187970,0.9,0.003175


Saved sweep: /kaggle/working/outputs/influence_edges_sweep.csv


In [15]:
def pick_eps(model, G, target=0.002, start=0.1):
    eps = start
    while eps <= 1.5:
        df = validate_influence_edges_sweep(model, G, eps_list=[eps], sign_tau=1e-5)
        if float(df["mean_abs_dA_meas"].iloc[0]) >= target:
            return eps
        eps *= 1.5
    return eps
eps_star = pick_eps(model, G)
print("Chosen eps:", eps_star)


Chosen eps: 0.1


In [18]:
# --- Fallback: rank edges by |∂A/∂g_mem| if edge_grad_rank isn't defined ---
def edge_grad_rank(model, G, head="A"):
    X, edges, E = to_features_device(G, requires_grad=True)
    if hasattr(E, "retain_grad"): E.retain_grad()
    with amp_context():
        A_hat, ddG_hat = model([(X, edges, E)])[0]
    model.zero_grad(set_to_none=True)
    (A_hat if head=="A" else ddG_hat).backward(retain_graph=True)
    g = E.grad[:, 2].detach().abs().cpu().numpy() if E.shape[1] >= 3 else None
    if g is None:
        return []
    order = g.argsort()[::-1]  # descending
    return order

# --- Ablation by edge-rank: bump g_mem on top vs bottom edges and measure ΔA ---
def ablate_edges_by_rank(model, G, k=10, eps=0.5):
    A0, _ = predict_graph_amp(model, G)
    order = edge_grad_rank(model, G, head="A")
    if len(order) == 0:
        print("No edge gradients available (E has <3 columns or no grad path).")
        return
    for label, idxs in [("top", order[:k]), ("bottom", order[-k:])]:
        H = clone_graph(G)
        XH, edgesH, EH = to_features_device(H)  # to align indices
        for eidx in idxs:
            i, j, _ = edgesH[eidx]
            H.edges[i, j]["g_mem"] = H.edges[i, j].get("g_mem", 0.0) + float(eps)
        A1, _ = predict_graph_amp(model, H)
        print(f"{label}  ΔA = {float(A1 - A0):.6f}")


In [19]:
# Pick the first graph (same one you used for scans)
uni, pdbid, chain, G = graphs[0]
print("Running ablation on:", uni, pdbid, chain)
ablate_edges_by_rank(model, G, k=10, eps=0.5)


Running ablation on: P00374 4M6J A
top  ΔA = -0.008354
bottom  ΔA = -0.006187
